In [1]:
import os
from pymilvus import MilvusClient

In [2]:
client = MilvusClient('../data/task_embeddings.db')
client.load_collection('swe_bench_verified')
client.load_collection('swe_bench_lite')

In [4]:
client.drop_index('swe_bench_verified', 'vector')
client.drop_index('swe_bench_lite', 'vector')
client.drop_index('swe_bench_verified', 'problem_statement_embeddings_lite')
client.drop_index('swe_bench_lite', 'problem_statement_embeddings')

In [5]:
client.describe_index('swe_bench_verified', 'problem_statement_embeddings')

In [216]:
index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    index_name='problem_statement_embeddings_lite',
    field_name="vector",
    index_type="FLAT",
    metric_type="COSINE", 
)
client.create_index('swe_bench_lite', index_params, sync=True)
# TODO Somehow this must have to do with working with two collections at once, maybe if I just load one?
#   Also I can move this into a Python file and try to step through Milvus?
index_params.add_index(
    index_name='problem_statement_embeddings_verified',
    field_name="vector",
    index_type="FLAT",
    metric_type="COSINE", 
)
client.create_index('swe_bench_verified', index_params, sync=True)

E20241217 16:20:28.279611 46050 collection_meta.cpp:148] [SERVER][CreateIndex][grpcpp_sync_ser] Add index failed, err: unrecognized token: ""
RPC error: [create_index], <MilvusException: (code=5, message=: internal error)>, <Time:{'RPC start': '2024-12-17 16:20:28.277461', 'RPC error': '2024-12-17 16:20:28.280696'}>
Failed to create an index on collection: swe_bench_verified


MilvusException: <MilvusException: (code=5, message=: internal error)>

In [218]:
client.describe_index('swe_bench_verified', 'problem_statement_embeddings_lite')

{'index_type': 'FLAT',
 'metric_type': 'COSINE',
 'dim': '768',
 'field_name': 'vector',
 'index_name': 'problem_statement_embeddings_lite',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [213]:
client.list_indexes(collection_name='swe_bench_verified')

['problem_statement_embeddings_lite']

In [201]:
swe_bench_lite = client.get(collection_name='swe_bench_lite', ids=[idx for idx in range(0, 53)])

In [202]:
swe_bench_verified = client.get(collection_name='swe_bench_verified', ids=[idx for idx in range(0, 500)])

In [203]:
swe_bench_lite_queries = [data['vector'] for data in swe_bench_lite]
swe_bench_verified_queries = [data['vector'] for data in swe_bench_lite]

In [204]:
swe_bench_verified[13]

{'id': 13,
 'instance_id': 'astropy__astropy-14539',
 'problem_statement': "`io.fits.FITSDiff` may sometimes report differences between identical files\n### Description\n\nIn some scenarios, `io.fits.FITSDiff` may report differences between identical files, even when comparing the same file to itself. This may be caused by improper handling of VLAs (variable-length arrays).\n\n### Expected behavior\n\n`io.fits.FITSDiff` only reports differences in files if they exist. Comparing a file to itself should never yield a difference.\n\n### How to Reproduce\n\n```python\r\nfrom astropy.io import fits\r\ncol = fits.Column('a', format='QD', array=[[0], [0, 0]])\r\nhdu = fits.BinTableHDU.from_columns([col])\r\nhdu.writeto('diffbug.fits', overwrite=True)\r\n\r\nprint(fits.FITSDiff('diffbug.fits', 'diffbug.fits').identical)\r\nfits.printdiff('diffbug.fits', 'diffbug.fits')\r\n\r\n```\r\nPrints out:\r\n```\r\nFalse\r\n fitsdiff: 5.2.1\r\n a: diffbug.fits\r\n b: diffbug.fits\r\n Maximum number of di

In [219]:
# TODO Try with a short problem statement, will I match with long ones too?
results = client.search(collection_name='swe_bench_lite', data=[swe_bench_verified[13]['vector']], limit=2000, search_params={"metric_type": "COSINE"})

In [220]:
results[0][0]

{'id': 43, 'distance': 0.9160262942314148, 'entity': {}}

Most similar result from the verified dataset to the first problem statement in the lite dataset.

In [221]:
print(client.get(collection_name='swe_bench_lite', ids=[results[0][0]['id']])[0]['problem_statement'])

napoleon_use_param should also affect "other parameters" section
Subject: napoleon_use_param should also affect "other parameters" section

### Problem
Currently, napoleon always renders the Other parameters section as if napoleon_use_param was False, see source
```
    def _parse_other_parameters_section(self, section):
        # type: (unicode) -> List[unicode]
        return self._format_fields(_('Other Parameters'), self._consume_fields())

    def _parse_parameters_section(self, section):
        # type: (unicode) -> List[unicode]
        fields = self._consume_fields()
        if self._config.napoleon_use_param:
            return self._format_docutils_params(fields)
        else:
            return self._format_fields(_('Parameters'), fields)
```
whereas it would make sense that this section should follow the same formatting rules as the Parameters section.

#### Procedure to reproduce the problem
```
In [5]: print(str(sphinx.ext.napoleon.NumpyDocstring("""\ 
   ...: Parameters 

The first problem statement in the lite dataset.

In [222]:
print(client.get(collection_name='swe_bench_verified', ids=[13])[0]['problem_statement'])

`io.fits.FITSDiff` may sometimes report differences between identical files
### Description

In some scenarios, `io.fits.FITSDiff` may report differences between identical files, even when comparing the same file to itself. This may be caused by improper handling of VLAs (variable-length arrays).

### Expected behavior

`io.fits.FITSDiff` only reports differences in files if they exist. Comparing a file to itself should never yield a difference.

### How to Reproduce

```python
from astropy.io import fits
col = fits.Column('a', format='QD', array=[[0], [0, 0]])
hdu = fits.BinTableHDU.from_columns([col])
hdu.writeto('diffbug.fits', overwrite=True)

print(fits.FITSDiff('diffbug.fits', 'diffbug.fits').identical)
fits.printdiff('diffbug.fits', 'diffbug.fits')

```
Prints out:
```
False
 fitsdiff: 5.2.1
 a: diffbug.fits
 b: diffbug.fits
 Maximum number of different data values to be reported: 10
 Relative tolerance: 0.0, Absolute tolerance: 0.0
Extension HDU 1:
   Data contains differences:

Most dissimilar result from the verified dataset to the first problem statement in the lite dataset.

In [223]:
print(client.get(collection_name='swe_bench_verified', ids=[results[0][-1]['id']])[0])

{'id': 49, 'instance_id': 'django__django-11451', 'problem_statement': 'ModelBackend.authenticate() shouldn\'t make a database query when username is None\nDescription\n\t\nIt\'s easier to explain my issue by adding a comment in the current implementation of ModelBackend.authenticate():\n\tdef authenticate(self, request, username=None, password=None, **kwargs):\n\t\tif username is None:\n\t\t\tusername = kwargs.get(UserModel.USERNAME_FIELD)\n\t\t# At this point, username and password can be None,\n\t\t# typically if credentials are provided for another backend.\n\t\t# Continuing makes a useless database query and runs\n\t\t# the password hasher needlessly (which is expensive).\n\t\ttry:\n\t\t\tuser = UserModel._default_manager.get_by_natural_key(username)\n\t\texcept UserModel.DoesNotExist:\n\t\t\t# Run the default password hasher once to reduce the timing\n\t\t\t# difference between an existing and a nonexistent user (#20760).\n\t\t\tUserModel().set_password(password)\n\t\telse:\n\t\t

In [149]:
results[0][-1]

{'id': 213, 'distance': 0.4491764008998871, 'entity': {}}